In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.tokenize import regexp_tokenize, word_tokenize, RegexpTokenizer
from nltk import pos_tag
from nltk import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, plot_confusion_matrix, f1_score
from sklearn.ensemble import RandomForestClassifier

In [2]:
df = pd.read_csv('data/judge-1377884607_tweet_product_company.csv',encoding='latin1')
df.rename(columns = {'emotion_in_tweet_is_directed_at': 'Product', 'is_there_an_emotion_directed_at_a_brand_or_product': 'Emotion', 'tweet_text': 'Tweet'}, inplace= True)
df.drop(9092, inplace=True)
df.reset_index(drop=True, inplace=True)

In [11]:
from nltk.tokenize import TweetTokenizer
tweet_tokenizer = TweetTokenizer()

In [27]:
df = pd.read_csv('data/judge-1377884607_tweet_product_company.csv',encoding='latin1')
df.rename(columns = {'emotion_in_tweet_is_directed_at': 'Product', 'is_there_an_emotion_directed_at_a_brand_or_product': 'Emotion', 'tweet_text': 'Tweet'}, inplace= True)
df.drop(9092, inplace=True)
df.reset_index(drop=True, inplace=True)
df['Text'] = df['Tweet'].str.lower()
df['Text'] = df['Text'].str.replace(r'[^\w\s]', '')
df['Text'] = df['Text'].str.split(' ')
df.dropna(subset=['Text'], inplace=True)
df['test'] = ''
df.fillna('na', inplace=True)
df.reset_index(drop= True, inplace=True)

In [28]:
apple_keywords = ['apple', '#apple', 'ipad', '#ipad', '#iphone', 'iphone', 'itunes', '#itunes', 'ios', 'airpods', '#ios', '#airpods'
                 'iphones', 'ipads', 'apples', 'quotapple', 'quotipad', 'ipadquot', 'quotipads', 'ipadsquot', 'applequot', 'quotiphone', 'iphonequot', 'quotios', 'iosquot', 'quotiphones', 'iphonesquot', 'ipad2']
google_keywords = ['google', '#google', 'android', '#android', 'googles', 'androids', 'quotgoogle', 'googlequot', 'quotandroid', 'androidquot', 'quotandroids', 'androidsquot' ]

for ind, val in enumerate(df['Text']):
    if df['Product'].values[ind] == 'na':
        for y in val:
            if y in apple_keywords:
                df['test'].values[ind] = y
            elif y in google_keywords:
                df['test'].values[ind] = y
                
    else:
        df['test'].values[ind] = df['Product'].values[ind]

In [29]:
df['test'] = df['test'].map(lambda x: x.lower())
df['test'] = df['test'].replace({'googles': 'google', 'apples': 'apple', 'androids': 'android','ipads': 'ipad', 'ipad2': 'ipad',
                                'quotgoogle': 'google', 'ipadquot': 'ipad', 'quotipad': 'ipad', 'androidquot': 'android', 'applequot': 'apple',
                               'googlequot': 'google', 'iphonequot': 'iphone', 'quotapple' : 'apple', 'iosquot': 'ios', 'quotiphone': 'iphone',
                                '#ipad' : 'ipad', '#google' : 'google', '#android': 'android', '#apple': 'apple', '#iphone': 'iphone'})
emotion_dict = {'No emotion toward brand or product' : 'Neutral emotion', "I can't tell": 'Neutral emotion', 'Positive emotion': 'Positive emotion',
               'Negative emotion': 'Negative emotion'}
df['Emotion'] = df['Emotion'].map(emotion_dict)


In [30]:
def company_search(test):
    if test == 'iphone':
        return 'apple'
    elif test == 'apple':
        return 'apple'
    elif test == 'ipad':
        return 'apple'
    elif test == 'ipad or iphone app':
        return 'apple'
    elif test == 'itunes':
        return 'apple'
    elif test == 'other apple product or service':
        return 'apple'
    elif test == 'ios':
        return 'apple'
    elif test == 'ipadquot':
        return 'apple'
    elif test == 'applequot':
        return 'apple'
    elif test == 'iphonequot':
        return 'apple'
    elif test == 'quotiphone':
        return 'apple'
    elif test == 'iosquot':
        return 'apple'
    elif test =='quotipad':
        return 'apple'
    elif test == 'quotapple':
        return 'apple'
    elif test =='google':
        return 'google'
    elif test == 'android':
        return 'google'
    elif test == 'other google product or service':
        return 'google'
    elif test == 'android app':
        return 'google'
    elif test == 'quotgoogle':
        return 'google'
    elif test == 'googlequot':
        return 'google'
    elif test == 'androidquot':
        return 'google'
    else:
        return 'unknown'
    
df['company'] = df['test'].apply(company_search)

In [31]:
target_dict = {'Negative emotion': 0, 'Neutral emotion': 1, 'Positive emotion': 2}
df['target'] = df['Emotion'].map(target_dict)

In [32]:
from collections import Counter
sw = stopwords.words('english')
from nltk.stem import WordNetLemmatizer
lemma = WordNetLemmatizer()

In [33]:
common_words = ['sxsw', 'mention', 'link', 'ipad', 'iphone', 'google', 'apple', '2', 'android', 'rt']
sw2 = sw.extend(common_words)

In [34]:
df['neg_target'] = df['target'].map({0:1, 1:0, 2:0})

In [35]:
df

,Tweet,Product,Emotion,Text,test,company,target,neg_target
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion,"[wesley83, i, have, a, 3g, iphone, after, 3, h...",iphone,apple,0,1
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion,"[jessedee, know, about, fludapp, , awesome, ip...",ipad or iphone app,apple,2,0
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion,"[swonderlin, can, not, wait, for, ipad, 2, als...",ipad,apple,2,0
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion,"[sxsw, i, hope, this, years, festival, isnt, a...",ipad or iphone app,apple,0,1
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion,"[sxtxstate, great, stuff, on, fri, sxsw, maris...",google,google,2,0
...,...,...,...,...,...,...,...,...
9086,"@mention Yup, but I don't have a third app yet...",na,Neutral emotion,"[mention, yup, but, i, dont, have, a, third, a...",android,google,1,0
9087,Ipad everywhere. #SXSW {link},iPad,Positive emotion,"[ipad, everywhere, sxsw, link]",ipad,apple,2,0
9088,"Wave, buzz... RT @mention We interrupt your re...",na,Neutral emotion,"[wave, buzz, rt, mention, we, interrupt, your,...",google,google,1,0
9089,"Google's Zeiger, a physician never reported po...",na,Neutral emotion,"[googles, zeiger, a, physician, never, reporte...",google,google,1,0


In [64]:
from imblearn.pipeline import Pipeline as imbpipe
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split

In [68]:
X = df.Text.str.join(' ')
y = df.neg_target

In [69]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y)

In [85]:
rfpipe = imbpipe(steps = [
    ('tfidf', TfidfVectorizer(max_features =1000, stop_words=sw, ngram_range = (1, 2))),
    ('sm', SMOTE()),
    ('rf', RandomForestClassifier())
])

In [71]:
mnbpipe = Pipeline(steps = [
    ('tfidf', TfidfVectorizer(max_features =500, stop_words=sw, ngram_range = (1, 2))),
    ('sm', SMOTE(sampling_strategy = .8)),
    ('mnb', MultinomialNB())
])

In [76]:
from sklearn.metrics import make_scorer

In [86]:
macro_recall = make_scorer(recall_score, average = 'macro')

In [88]:
cv = cross_validate(rfpipe, X_train, y_train, scoring = macro_recall)

In [78]:
from sklearn.model_selection import cross_val_predict

In [91]:
y_hat_train = cross_val_predict(mnbpipe, X_train, y_train)

In [92]:
confusion_matrix(y_train, y_hat_train)

array([[5420,  971],
       [ 215,  212]])

In [93]:
y_hat1 = cross_val_predict(mnbpipe, X, y)

In [94]:
confusion_matrix(y, y_hat1)

array([[7099, 1422],
       [ 292,  278]])